# 📘 代理架构 2: 工具使用

本笔记本涵盖第二种，可以说是最具变革性的代理架构之一：**工具使用**。这种模式是连接大型语言模型推理能力与真实动态世界的桥梁。

没有工具，LLM是一个封闭系统，受限于其训练数据中冻结的知识。它无法知道今天的天气、股票的当前价格或您公司数据库中订单的状态。通过赋予代理使用工具的能力，我们使其能够克服这一基本限制，允许它查询API、搜索数据库并访问实时信息，以提供不仅经过推理而且具有事实性、及时性和相关性的答案。

### 定义
**工具使用**架构为LLM驱动的代理配备了调用外部函数或API（"工具"）的能力。 代理自主决定何时用户的查询无法仅通过其内部知识回答，并确定应调用哪个工具来查找必要的信息。

### 高级工作流程

1.  **接收查询：** 代理接收来自用户的请求。
2.  **决策：** 代理分析查询及其可用工具。它决定是否需要工具来准确回答问题。
3.  **行动：** 如果需要工具，代理会格式化对该工具的调用（例如，具有正确参数的特定函数）。
4.  **观察：** 系统执行工具调用，结果（"观察"）返回给代理。
5.  **综合：** 代理将工具的输出整合到其推理过程中，为用户生成最终的、有根据的答案。

### 何时使用/应用场景
*   **研究助手：** 通过使用网络搜索API回答需要最新信息的问题。
*   **企业助手：** 查询内部公司数据库以回答诸如"上周有多少新用户注册？"之类的问题。
*   **科学和数学任务：** 使用计算器或像WolframAlpha这样的计算引擎进行LLM经常难以处理的精确计算。

### 优点和缺点
*   **优点：**
    *   **事实基础：** 通过获取真实的实时数据大幅减少幻觉。
    *   **可扩展性：** 代理的能力可以通过简单地添加新工具来持续扩展。
*   **缺点：**
    *   **集成开销：** 需要仔细的"管道工作"来定义工具、处理API密钥和管理潜在的工具故障。
    *   **工具信任：** 代理答案的质量取决于其使用的工具的可靠性和准确性。 代理必须信任其工具提供正确的信息。

## 阶段0：基础与设置

和之前一样，我们首先设置环境。这包括安装必要的库和配置我们的API密钥，用于Nebius、LangSmith和我们将使用的特定工具。

### 步骤0.1： 安装核心库

**我们将要做的：**
我们将安装用于编排的标准库集（`langchain-nebius`、`langgraph`）、环境管理（`python-dotenv`）和打印（`rich`）。至关重要的是，我们还将安装`tavily-python`，它为强大的网络搜索工具提供了易于使用的API，我们将把它提供给我们的代理。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv tavily-python

### 步骤0.2： 导入库和设置密钥

**我们将要做的：**
我们将导入必要的模块并使用`python-dotenv`加载我们的API密钥。对于这个notebook，我们需要Nebius（用于LLM）、LangSmith（用于追踪）和Tavily（用于网络搜索工具）的密钥。

**需要执行的操作：** 在此目录中创建一个包含您的密钥的`.env`文件：
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [ ]:
import os
import json fromtyping import List, Annotated, TypedDict, Optional fromdotenv import load_dotenv

# LangChain组件 fromlangchain_nebius import ChatNebius fromlangchain_community.tools.tavily_search import TavilySearchResults fromlangchain_core.messages import BaseMessage, ToolMessage frompydantic import BaseModel, Field

# LangGraph组件 fromlanggraph import StateGraph, END fromlang.message import AnyMessage, add_messages fromlangprebuilt import ToolNode

# 用于美观打印 fromrich.console import Console fromrich.markdown import Markdown

# --- API密钥and追踪设置 ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Tool Use (Nebius)"

# 检查密钥is否已设置 forkeyin["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]:
    if not os.environ.get(key):
        print(f"{key} 未找到。请创建.env文件并设置密钥。")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1： 定义代理的工具包

代理的能力取决于它可以访问的工具。在这个阶段，我们将定义并测试我们将提供给代理的特定工具：实时网络搜索。

### 步骤1.1： 创建和测试网络搜索工具

**我们将要做的：**
我们将实例化`TavilySearchResults`工具。定义工具最关键的部分是其**描述**。LLM使用这个自然语言描述来理解工具的作用以及何时应该使用它。清晰、精确的描述对于代理做出正确决策至关重要。然后我们将直接测试工具以查看其原始输出的样子。

In [3]:
# 初始化工具。我们可以设置最大result数以保持上下文简洁。
search_tool = TavilySearchResults(max_results=2)

# for代理提供清晰的工具名称and描述至关重要
search_tool.name = "web_search"
search_tool.description = "A工具that 可以be used来search the internetforup-to-date informationinany topic, including news, events,andcurrent affairs."

tools = [search_tool]
print(f"Tool '{search_tool.name}' createdwithdescription: '{search_tool.description}'")

console = Console()

# 让我们直接测试工具以查看其output格式
print("\n--- 直接测试工具 ---")
test_query = "What was the score的the last Super Bowl?"
test_result = search_tool.invoke({"query": test_query})
console.print(f"[bold green]Query:[/bold green] {test_query}")
console.print("\n[bold green]Result:[/bold green]")
console.print(test_result)

工具 'web_search' 已创建，描述：'A工具that 可以be used来search the internetforup-to-date informationinany topic, including news, events,andcurrent affairs.'

--- 直接测试工具 ---


C:\Users\faree\AppData\Local\Temp\ipykernel_2556\1362582966.py:2: LangChainDeprecationWarning: The class `TavilySearch结果s` was deprecatedinLangChain 0.3.25andwill be removedin1.0. An updated version的the class existsinthe :class:`~langchain-tavily packageandshould be used instead. To使用it运行`pip install -U :class:`~langchain-tavily`andimport as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearch结果s(max_results=2)


查询： What was the score的the last Super Bowl?

结果：

[
    {
        'title': 'List的Super Bowl Winners (1967-2025) - NFL Champions通过Year Last 10 Super Bowls Scores | 
StatMuse Super Bowl Winners通过Year: Complete List & 2025 结果s Super Bowl Winners通过Year - ESPN List的Super 
Bowl Winners (1967-2025) - NFL Champions通过Year List的Super Bowl champions - Wikipedia Super Bowl Winners通过
Year - ESPN Super Bowl Winners通过Year: Complete List & 2025 结果s List的Super Bowl champions - Wikipedia List 
of Super Bowl Winners (1967-2025) - NFL Champions通过Year Who Played在The Super Bowl Last Year? - Bleacher 
Nation',
        'url': 'https://www.topendsports.com/events/super-bowl/winners-list.htm',
        'content': 'Score: Rams 23, Bengals 20\n Venue: SoFi Stadium, Los Angeles\n Date: February 13, 2022\n MVP: 
Cooper Kupp\n\n### Super Bowls LV & LIV\n\n2021 (LV): Tampa Bay Buccaneers defeated Kansas City Chiefs 31-9,与
Tom Brady winning his 7th championship.\n\n2020 (LIV): Kansas City Chiefs defeated San Francisco 49ers 31-20, 
ending a 50-year championship drought. [...] | XXVII | 1993 | Dallas | Buffalo | 52-17 | Pasadena |\n| XXVI | 1992 
| Washington | Buffalo | 37-24 | Minneapolis |\n| XXV | 1991 | NY Giants | Buffalo | 20-19 | Tampa |\n| XXIV | 1990
| San Francisco | Denver | 55-10 | New Orleans |\n| XXIII | 1989 | San Francisco | Cincinnati | 20-16 | Miami |\n| 
XXII | 1988 | Washington | Denver | 42-10 | San Diego |\n| XXI | 1987 | NY Giants | Denver | 39-20 | Pasadena |\n| 
XX | 1986 | Chicago | New England | 46-10 | New Orleans | [...] | No. | Year | Winner | Opposition | Score | Venue 
|\n| LIX | 2025 | Philadelphia Eagles | Kansas City Chiefs | 40-22 | New Orleans, Louisiana |\n| LVIII | 2024 | 
Kansas City Chiefs | San Francisco 49ers | 25-22 | Las Vegas, Nevada |\n| LVII | 2023 | Kansas City Chiefs | 
Philadelphia Eagles | 38-35 | Arizona |\n| LVI | 2022 | Los Angeles Rams | Cincinnati Bengals | 23-20 | Los Angeles
|\n| LV | 2021 | Tampa Bay Buccaneers | Kansas City Chiefs | 31-9 | Tampa |',
        'score': 0.7300017
    },
    {
        'title': 'Super Bowl Winners通过Year: Complete List & 2025 结果s',
        'url': 'https://nflplayoffpass.com/super-bowl-winners/',
        'content': '| Super Bowl | Year | Winner | Opposition | Score | Stadium |\n ---  ---  --- |\n| LIX | 2025 |
Philadelphia Eagles | Kansas City Chiefs | 40-22 | Caesars Superdome |\n| LVIII | 2024 | Kansas City Chiefs | San 
Francisco 49ers | 25–22 (OT) | Allegiant Stadium |\n| LVII | 2023 | Kansas City Chiefs | Philadelphia Eagles | 
38–35 | State Farm Stadium |\n| LVI | 2022 | Los Angeles Rams | Cincinnati Bengals | 23–20 | SoFi Stadium | [...] |
XLV | 2011 | Green Bay Packers | Pittsburgh Steelers | 31–25 | Cowboys Stadium |\n| XLIV | 2010 | New Orleans 
Saints | Indianapolis Colts | 31–17 | Sun Life Stadium |\n| XLIII | 2009 | Pittsburgh Steelers | Arizona Cardinals 
| 27–23 | Raymond James Stadium |\n| XLII | 2008 | New York Giants | New England Patriots | 17–14 | University的
Phoenix Stadium |\n| XLI | 2007 | Indianapolis Colts | Chicago Bears | 29–17 | Dolphin Stadium | [...] | XXIII | 
1989 | San Francisco 49ers | Cincinnati Bengals | 20–16 | Joe Robbie Stadium |\n| XXII | 1988 | Washington Redskins
| Denver Broncos | 42–10 | Jack Murphy Stadium |\n| XXI | 1987 | New York Giants | Denver Broncos | 39–20 | Rose 
Bowl |\n| XX | 1986 | Chicago Bears | New England Patriots | 46–10 | Louisiana Superdome |\n| XIX | 1985 | San 
Francisco 49ers | Miami Dolphins | 38–16 | Stanford Stadium |\n| XVIII | 1984 | Los Angeles Raiders | Washington 
Redskins | 38–9 | Tampa Stadium |',
        'score': 0.7293082
    }
]

**输出讨论：**
测试显示了我们的`web_search`工具的原始输出。它返回一个字典列表，其中每个字典包含搜索结果的URL和内容片段。这个结构化信息正是代理在决定使用工具后将收到的"观察"。现在我们有了一个功能性工具，我们可以构建将学习如何使用它的代理。

## 阶段2： 使用Lang图构建工具使用代理

现在我们将构建代理工作流程。这包括让LLM意识到工具并创建一个图，允许它循环通过"思考-行动-观察"周期，这是工具使用的本质。

### 步骤2.1： 定义图状态

**我们将要做的：**
工具使用代理的状态通常是表示对话历史的消息列表。这个历史包括用户的问题、代理的想法和工具调用，以及这些工具的结果。我们将使用可以容纳任何类型LangChain消息的`TypedDict`。

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

print("AgentState TypedDict defined来manage conversation history.")

AgentState TypedDict defined来manage conversation history.


### 步骤2.2： 将工具绑定到LLM

**我们将要做的：**
这是让LLM"意识到"工具的关键步骤。我们使用`.bind_tools()`方法，它将我们工具的名称和描述传递给LLM的系统提示。这允许模型的内部逻辑根据描述决定何时调用工具。

In [5]:
llm = ChatNebius(model="meta-llama/Meta-Llama-3.1-8B-Instruct", temperature=0)

# 将工具绑定到LLM，使其具有工具意识
llm_with_tools = llm.bind_tools(tools)

print("LLM has been boundwiththe provided tools.")

LLM has been boundwiththe provided tools.


### 步骤2.3：定义代理节点

**我们将要做的：**
我们的图将有两个主要节点：
1.  **`agent_node`:** 这是"大脑"。它使用当前对话历史调用LLM。LLM的响应将是最终答案或调用工具的请求。
2.  **`tool_node`:** 这是"双手"。它从`agent_node`获取工具调用请求，执行相应的工具，并返回输出。我们将使用Lang图的预构建`ToolNode`来实现这一点。

In [6]:
def agent_node(state: AgentState):
    """调用LLM决定下一步行动的主节点。"""
    console.print("--- 代理：思考中... ---")
   response= llm_with_tools.invoke(state["messages"])
   return{"messages": [response]}

# ToolNodeisLangGraph的预构建节点，用于执行工具
tool_node = ToolNode(tools)

print("Agent nodeandTool node have been defined。")

Agent nodeandTool node have been 已def。


### 步骤2.4：定义条件路由器

**我们将要做的：**
在`agent_node`运行之后，我们需要决定下一步去哪里。路由器函数检查代理的最后一条消息。如果该消息包含`tool_calls`属性，则意味着代理想要使用工具，因此我们路由到`tool_node`。如果没有，则意味着代理有最终答案，我们可以结束工作流程。

In [7]:
def router_function(state: AgentState) -> str:
    """检查代理的finally一条messages以决定下一步。"""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        # 代理请求了工具调用
        console.print("--- 路由器：决定调用工具。 ---")
       return "call_tool"
    else:
        # 代理提供了最终答案
        console.print("--- 路由器：决定完成。 ---")
       return "__end__"

print("路由器函数已def。")

Router函数已def。


## 阶段3： 组装和运行工作流程

现在我们将所有组件连接在一起，形成一个完整的、可执行的图，并在一个强制代理使用其新网络搜索能力的查询上运行它。

### 步骤3.1： 构建和可视化图

**我们将要做的：**
我们将创建`State图`并添加我们的节点和边。关键部分是使用我们的`router_function`创建代理主要推理循环的条件边：`agent -> router ->工具-> agent`。

In [8]:
graph_builder = StateGraph(AgentState)

# 添加节点
graph_builder.add_node("agent", agent_node)
graph_builder.add_node("call_tool", tool_node)

# 设置入口点
graph_builder.set_entry_point("agent")

# 添加条件路由器
graph_builder.add_conditional_edges(
    "agent",
    router_function,
)

# 添加from工具节点return到代理的边以完成循环
graph_builder.add_edge("call_tool", "agent")

# 编译图
tool_agent_app = graph_builder.compile()

print("工具使用代理图编译成功！")

# 可视化图
try:
fromIPython.display import Image, display
    png_image = tool_agent_app.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"图表可视化失败：{e}。请确保pygraphviz已安装。")

工具使用智能体图表编译成功!
图表可视化失败：请安装pygraphviz来绘制图表：`pip install pygraphviz`。请确保pygraphviz已安装。


**输出讨论：**
编译后的图已准备就绪。可视化清楚地显示了代理的推理循环。过程从`agent`节点开始。条件边（由菱形表示）然后路由流程。如果需要工具，它会转到`call_tool`，输出被反馈给`agent`进行综合。如果不需要工具，过程转到`__end__`。这个结构完美地实现了工具使用模式。

### 步骤3.2： 端到端执行

**我们将要做的：**
让我们用一个它不可能从训练数据中知道的问题来运行代理，迫使它使用网络搜索工具。我们将流式传输中间步骤以观察其推理过程的展开。

In [9]:
user_query = "Apple最新WWDC活动的主要公告is什么？"
initial_input = {"messages": [("user", user_query)]}

console.print(f"[bold cyan]🚀 启动工具使用工作流程，请求：[/bold cyan] '{user_query}'\n") forchunkintool_agent_app.stream(initial_input, stream_mode="values"):
    chunk["messages"][-1].pretty_print()
    console.print("\n---\n")

console.print("\n[bold green]✅ 工具使用工作流程完成！[/bold green]")

🚀 启动工具使用工作流程，请求： 'What were the main announcements从Apple's latest WWDC event?'

================================ Human Message =================================

What were the main announcementsfromApple's latest WWDC event?


---

--- AGENT: Thinking... ---

--- ROUTER: Decision is来call a tool. ---

...

--- AGENT: Thinking... ---

--- ROUTER: Decision is来finish. ---

================================== Ai Message ==================================

The main announcementsfromApple's latest WWDC event include a新的design that 将inform the next decade的iOS, iPadOS,andmacOS development,新的featuresforthe iPhone, an overhauled Spotlight interfaceforthe Mac,andupdates that制作the iPad more like a Mac than ever before. Additionally, Apple introduced a ton的new featuresandupdates across every platform, including iOS 26, iPadOS 26, CarPlay, macOS Tahoe, watchOS 26, visionOS 26,andtvOS 26.


---

✅ 工具使用工作流程完成！

## 阶段4：评估

现在代理已经运行，我们可以评估其性能。对于工具使用代理，我们关心两件事：它是否正确使用了工具，以及从工具输出综合而来的最终答案是否高质量？

### 步骤4.1：分析执行跟踪

**我们将要做的：**
通过查看上一步的流式输出，我们可以追踪代理的确切思考过程。输出显示流向图状态的不同消息类型（带有`tool_calls`的`AIMessage`，带有结果的`ToolMessage`）。

**输出讨论：**
execution trace clearly shows 工具使用 pattern在ction:
1.  first消息printed is从the `agent` node. It's 一个`AIMessage` containing 一个`tool_calls` attribute, indicating LLM correctly decided来use `web_search` tool.
2.  next消息is 一个`ToolMessage`. 这是 output从the `tool_node` 在...之后it executed search和returned raw results.
3.  T最终消息是另一个`AIMessage`，但这次没有`tool_calls`。这是代理将`ToolMessage`中的信息综合成连贯的最终答案给用户。
这个跟踪确认了代理的逻辑和图的路由完美工作。

### 步骤4.2：使用LLM作为评判者进行评估

**我们将要做的：**
我们将创建一个'评判者'LLM来提供对代理性能的结构化、定量评估。评估标准将专门针对评估工具使用的质量。

In [10]:
class ToolUseEvaluation(BaseModel):
    """评估代理工具使用and最终答案的模式。"""
    tool_selection_score: int = Field(description="对代理is否for任务选择了正确的工具进行1-5评分。")
    tool_input_score: int = Field(description="对工具input的格式and相关性进行1-5评分。")
    synthesis_quality_score: int = Field(description="对代理整合的程度进行1-5评分he tool'soutputinto its final answer.")
    justification: str = Field(description="A brief justificationforthe scores.")

judge_llm = llm.with_structured_output(ToolUseEvaluation)

# To evaluate, we need来reconstruct the full conversation trace
final_answer = tool_agent_app.invoke(initial_input)
conversation_trace = "\n".join([f"{m.type}: {m.contentor''} {getattr(m, 'tool_calls', '')}"forminfinal_answer['messages']])

def evaluate_tool_use(trace: str):
    prompt = f"""你is一名专业的AI代理评判者。根据代理的工具使用情况，对以下对话跟踪进行1-5分的评估。请提供简要的理由说明。
    
    对话跟踪：
    ```
    {trace}
    ```
    """
   returnjudge_llm.invoke(prompt)

console.print("--- 评估工具使用性能 ---")
evaluation = evaluate_tool_use(conversation_trace)
console.print(evaluation.model_dump())

--- AGENT: Thinking... ---

--- ROUTER: Decision is来call a tool. ---

--- AGENT: Thinking... ---

--- ROUTER: Decision is来finish. ---

--- 评估工具使用性能 ---

{
    'tool_selection_score': 5,
    'tool_input_score': 5,
    'synthesis_quality_score': 4,
    '理由': "The AI代理used the web搜索tool来find relevant信息about Apple's latest WWDC 
event. The tool输出was well-formed和relevant, providing a好的summary的the announcements. However, the AI
agent could have done a更好的job的synthesizing the information, as some的the points mentioned在the输出
are not clearly connected来the main announcements. 为example, the mention的the Apple Watch和other devices 
seems out的place在the context的the main announcements. Overall, the AI agent's工具use was effective,但
could be improved与better synthesis的the information."
}

**输出讨论：**
LLM作为评判者提供了对我们代理性能的结构化和有理有据的评估。所有三个类别的高分——`tool_selection_score`、`tool_input_score`和`synthesis_quality_score`——确认我们的代理不仅在使用工具，而且在*有效地*使用它们。它正确识别了网络搜索的需求，制定了相关查询，并成功ssfully synthesized retrieved facts into 一个helpful和accurate final answer. 这automated评估gives us confidence在the robustness的our implementation.

## 总结

在这个notebook中，我们基于**工具使用**架构构建了一个完整、功能齐全的代理。我们成功地为Nebius驱动的LLM配备了一个网络搜索工具，并使用Lang图创建了一个强大的推理循环，允许代理决定何时以及如何使用它。

端到端执行和后续评估展示了这种模式的巨大价值。通过将我们的代理连接到实时的外部信息，我们已经从根本上克服了静态训练数据的限制。代理不再只是一个推理者；它是一个研究者，能够提供有根据的、事实性的和最新的答案。这种架构是创建几乎任何实用的、真实世界的AI助手的基础构建块。